In [1]:
import torch
from torch.utils.data import DataLoader

from aurora import Aurora

from bfm_finetune.aurora_mod import AuroraFlex
from bfm_finetune.plots import plot_eval
from bfm_finetune.dataloaders.geolifeclef_species.dataloader import GeoLifeCLEFSpeciesDataset
from bfm_finetune.dataloaders.dataloader_utils import custom_collate_fn
from bfm_finetune.utils import load_checkpoint, seed_everything

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

seed_everything(42)

/home/tkhan/bfm-finetune/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
STORAGE_DIR: /projects/prjs1134/data/projects/biodt/storage


In [2]:
base_model = Aurora(use_lora=False) # stabilise_level_agg=True
base_model.load_checkpoint("microsoft/aurora", "aurora-0.25-pretrained.ckpt", strict=False) # strict=False
atmos_levels = (50, 100, 150, 200, 250, 300, 400, 500, 600, 700, 850, 925, 1000)
base_model.to(device)

num_species = 500

model = AuroraFlex(base_model=base_model, in_channels=num_species, hidden_channels=160,
                    out_channels=num_species, atmos_levels=atmos_levels)
model.to(device)

val_dataset = GeoLifeCLEFSpeciesDataset(num_species=num_species, mode="val")
val_dataloader = DataLoader(
        val_dataset,
        batch_size=1,
        shuffle=False,
        collate_fn=custom_collate_fn,
        num_workers=1,
    )

params_to_optimize = model.parameters()
optimizer = torch.optim.AdamW(params_to_optimize, lr=1.0)

Initialized AuroraFlex Mod
files 1


In [3]:
PATH = "/home/tkhan/bfm-finetune/outputs/2025-04-08/11-12-37/checkpoints/best_checkpoint.pth"
#_, _ = load_checkpoint(model, optimizer, PATH)
try:
    checkpoint = torch.load(PATH, map_location=device)
    model.load_state_dict(checkpoint["model_state_dict"], strict=False)
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
except Exception as e:
    print("No valid checkpoint loaded:", e)


for sample in val_dataloader:
    batch = sample["batch"].to(device)
    target = sample["target"]
    with torch.inference_mode():
        prediction = model.forward(batch)
    plot_eval(
        batch=batch,
        prediction_species=prediction,
        out_dir="plots_dir",
        save=False
    )

No valid checkpoint loaded: Error(s) in loading state_dict for AuroraFlex:
	size mismatch for base_model.encoder.atmos_latents: copying a param with shape torch.Size([3, 256]) from checkpoint, the shape in current model is torch.Size([3, 512]).
	size mismatch for base_model.encoder.surf_level_encoding: copying a param with shape torch.Size([256]) from checkpoint, the shape in current model is torch.Size([512]).
	size mismatch for base_model.encoder.surf_mlp.net.0.weight: copying a param with shape torch.Size([1024, 256]) from checkpoint, the shape in current model is torch.Size([2048, 512]).
	size mismatch for base_model.encoder.surf_mlp.net.0.bias: copying a param with shape torch.Size([1024]) from checkpoint, the shape in current model is torch.Size([2048]).
	size mismatch for base_model.encoder.surf_mlp.net.2.weight: copying a param with shape torch.Size([256, 1024]) from checkpoint, the shape in current model is torch.Size([512, 2048]).
	size mismatch for base_model.encoder.surf_ml

RuntimeError: shape '[1, 5, 13, 2, 721, 1440]' is invalid for input of size 6323200